In [1]:

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sys
import itertools
from itertools import islice
from itertools import product
import pyspark
from pyspark.sql import functions as F
from pyspark.sql import Window, DataFrame
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import udf
import pyspark.sql.functions as F
from pyspark.sql.functions import col, row_number, expr,explode
from pyspark.ml.tuning import CrossValidator,ParamGridBuilder
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import FloatType, IntegerType, LongType, StringType,StructType, StructField
from pyspark.sql import Row
from pyspark.ml.feature import StringIndexer
from google.cloud import storage
from pyspark.storagelevel import StorageLevel
from pyspark.ml.feature import StringIndexerModel
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.mllib.evaluation import RegressionMetrics, RankingMetrics
from pyspark.ml.recommendation import ALSModel
from pyspark.ml.feature import IndexToString
import os
import time


# Tạo session mới
print("Creating new Spark session...")

spark = SparkSession.builder \
    .appName("MongoDb") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .config("spark.executor.memory", "15g") \
    .config("spark.mongodb.input.uri", "mongodb://192.168.0.103:27017/shopDev.Shops") \
    .config("spark.mongodb.output.uri", "mongodb://192.168.0.103:27017/shopDev.Shops") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .getOrCreate()

print("Spark master:", spark.sparkContext.master)


Creating new Spark session...
Spark master: yarn


25/05/12 13:20:06 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
# spark.stop()
conf = dict(spark.sparkContext.getConf().getAll())

print("spark.executor.instances:", conf.get("spark.executor.instances"))
print("spark.executor.cores:", conf.get("spark.executor.cores"))
print("spark.executor.memory:", conf.get("spark.executor.memory"))
print("spark.driver.memory:", conf.get("spark.driver.memory"))
print("spark.sql.shuffle.partitions:", conf.get("spark.sql.shuffle.partitions"))
print("spark.kryoserializer.buffer.max:", conf.get("spark.kryoserializer.buffer.max"))
print("spark.kryoserializer.buffer:", conf.get("spark.kryoserializer.buffer"))


spark.executor.instances: 3
spark.executor.cores: 4
spark.executor.memory: 25g
spark.driver.memory: 25g
spark.sql.shuffle.partitions: 100
spark.kryoserializer.buffer.max: 1024m
spark.kryoserializer.buffer: 64m


In [3]:
bucket_name = "team-15"

800

In [5]:
model = ALSModel.load(f"gs://{bucket_name}/als_model")

In [6]:
item_factors = model.itemFactors


In [7]:
item_factors = model.itemFactors.withColumnRenamed("id", "item_index")


In [8]:
from pyspark.sql.functions import col

# Cast sang int để khớp với itemFactors
df_items = df_spark.withColumn("item_index", col("item_index").cast("int"))


In [9]:
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
import numpy as np

# Hàm tính cosine similarity
def cosine_sim(a, b):
    a = np.array(a)
    b = np.array(b)
    return float(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)))

cosine_sim_udf = udf(cosine_sim, FloatType())


target_items = item_factors.join(df_items, on="item_index", how="inner")


item_pairs = target_items.alias("a").crossJoin(item_factors.alias("b")) \
    .filter(col("a.item_index") != col("b.item_index")) \
    .withColumn("similarity", cosine_sim_udf(col("a.features"), col("b.features")))

In [10]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

windowSpec = Window.partitionBy("a.item_index").orderBy(col("similarity").desc())

top_similar_items = item_pairs \
    .withColumn("rank", row_number().over(windowSpec)) \
    .filter(col("rank") <= 5) \
    .select(
        col("a.item_index").alias("item"),
        col("b.item_index").alias("similar_item"),
        "similarity"
    )

top_similar_items.show(20)


In [11]:
top_similar_items.write.mode("overwrite").csv(f"gs://{bucket_name}/item_based")